In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import string
import sys
import re
import emoji
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize, MWETokenizer
from pyvi import ViTokenizer
from unicodedata import normalize
pd.set_option('max_colwidth', -1)
from hdfs import InsecureClient
import pickle

/home/polynote/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  from ipykernel import kernelapp as app


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import hamming_loss,  accuracy_score, multilabel_confusion_matrix, f1_score

In [3]:
client = InsecureClient('http://mt02.up.vn:50070')

In [4]:
def read_csv_hdfs(path_file, sep):
    with client.read(path_file, encoding='utf-8') as reader:
        df_ = pd.read_csv(reader, delimiter=sep)
        return df_

In [5]:
path_file_02 = '/user/anhdt157/politic_fb_post_clean/2006/part-00000-924c1669-1b15-40c8-9f9b-99747f77ea9c-c000.csv'

In [6]:
df_test = read_csv_hdfs(path_file_02, '|')

# Xử lý dữ liệu bị trùng 

In [7]:
df_test['clean_content']=df_test['clean_content'].astype(str)

In [8]:
def clean_dup(text):
    return re.sub(r'(_) ','',text)

In [9]:
df_test['clean_content'] = df_test['clean_content'].apply(lambda x: clean_dup(x))

In [10]:
def remove_first50w(text):
    text = text.split()[:50]
    return ' '.join(text)
def remove_last50w(text):
    text = text.split()[-50:]
    return ' '.join(text)

In [11]:
df_new = df_test[0:500000]
df_new['remove_first50w'] = df_new['clean_content'].apply(lambda x: remove_first50w(x))
df_new['remove_last50w'] = df_new['clean_content'].apply(lambda x: remove_last50w(x))
df_new = df_new.drop_duplicates(subset=['remove_first50w'], keep ='first')
df_new = df_new.drop_duplicates(subset=['remove_last50w'], keep ='first')

/home/polynote/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/polynote/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# Gán nhãn cho dữ liệu 

In [12]:
df_keyword = pd.read_csv('keyword_label.txt',sep = '|',error_bad_lines=False,encoding='utf-8')

In [13]:
labels = df_keyword['label'].tolist()
keywords = df_keyword['clean_key'].apply(lambda x: x.split(",")).tolist()
thresholds = df_keyword['threshold_keyword'].to_list()
except_keywords = df_keyword['keyword_except'].apply(lambda x: x.split(",")).to_list()

In [14]:
vocabulary = ','.join(df_keyword['clean_key']) + ','.join(df_keyword['keyword_except'])
vocabulary = list({vocabulary})
vectorizer = CountVectorizer(binary=True, ngram_range=(1,5))
vectorizer.fit(vocabulary)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 5), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [15]:
def labeling(df,labels, keywords,thresholds,except_keywords):
    '''Gán nhãn cho bài viết : bài viết xuất hiện số từ trong tập keyword > threshold và không xuất hiện
    từ trong nhóm keyword_except'''
    test = vectorizer.transform(df)
    data = pd.DataFrame.sparse.from_spmatrix(test)
    data.columns = vectorizer.get_feature_names()
    for label, keyword,threshold,except_keyword in zip(labels, keywords,thresholds,except_keywords):
        data['keyword'] = data.loc[:,data.columns.intersection(keyword)].sum(axis=1)
        data['keyword_except'] = data.loc[:,data.columns.intersection(except_keyword)].sum(axis=1)
        data['sum_keyword'] = data['keyword'] - 1000*data['keyword_except']
        data[label] = data['sum_keyword'].apply(lambda x:1 if x >= threshold else 0)
        label_result = data[data.columns.intersection(labels)]
    return label_result

In [16]:
%%time
''' Gán nhãn dữ liệu. data final là tập tữ liệu đã được gán nhãn, sum_label là tổng số nhãn được
gán của bài vieets'''
df_labeling = labeling(df_new['clean_content'], labels, keywords,thresholds,except_keywords)
df_new.reset_index(drop=True, inplace=True)
df_labeling.reset_index(drop=True, inplace=True)
data_final = pd.concat([df_new,df_labeling],axis=1)
data_final['sum_label']=data_final.iloc[:,8:20].sum(axis=1)

CPU times: user 5min 10s, sys: 10.4 s, total: 5min 20s
Wall time: 5min 21s


# Training model

In [17]:
''' Chỉ train cho tập dữ liệu được gán ít nhất 1 nhãn'''
df_train = data_final.loc[data_final['sum_label']>0]

In [18]:
X = df_train['clean_content']
y = df_train.iloc[:,8:20]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [19]:
%%time
tfidf = TfidfVectorizer()
tfidf.fit(X)
tfidf_train = tfidf.transform(x_train)
tfidf_test = tfidf.transform(x_test)

CPU times: user 46.9 s, sys: 880 ms, total: 47.7 s
Wall time: 47.8 s


In [20]:
%%time
classify = OneVsRestClassifier(XGBClassifier(learning_rate=0.1,max_depth=3 ))
classify.fit(tfidf_train,y_train)
y_pred = classify.predict(tfidf_test)

CPU times: user 1h 44min 21s, sys: 30.5 s, total: 1h 44min 51s
Wall time: 5min 15s


In [21]:
pickle.dump(classify, open('model_train.pkl','wb'))
pickle.dump(tfidf, open('tfidt_train.pkl','wb'))
pickle.dump(vectorizer, open('vectorize_keyword_train.pkl','wb'))

In [22]:
print("hamming loss {}".format(hamming_loss(y_test,y_pred)))
print("accuracy {}".format(accuracy_score(y_test,y_pred)))
print("micro avg {}".format(f1_score(y_test,y_pred, average='micro')))
print("macro avg {}".format(f1_score(y_test,y_pred, average='macro')))

hamming loss 0.016454974759811107
accuracy 0.8243282852955545
micro avg 0.92431845416417
macro avg 0.9148980450881478


In [23]:
print(multilabel_confusion_matrix(y_test,y_pred))

[[[19641    28]
  [   44   757]]

 [[18408    96]
  [  153  1813]]

 [[17252   120]
  [  196  2902]]

 [[16767   193]
  [  832  2678]]

 [[17541    14]
  [  177  2738]]

 [[20028     9]
  [   47   386]]

 [[17173    62]
  [  613  2622]]

 [[13807   325]
  [  743  5595]]

 [[20024    47]
  [   65   334]]

 [[16367    31]
  [   43  4029]]

 [[19592    43]
  [  123   712]]

 [[20315    19]
  [   19   117]]]


In [24]:
a = 'infor choi jin hyuk are coming back as a zombie mong_chờ tạo_hình xác sống của jin hyuk quá đi choi jin hyuk và park_joo hyun sẽ tham_gia cho một bộ phim zombie mới trên kbs có tên là thám_tử zombie zombie dectective bộ phim kể về một thây_ma sống lại năm thứ hai và trở_thành thám_tử tìm về quá_khứ cố hết_sức để cùng tồn_tại với con_người choi jin hyuk sẽ vào_vai kim moo young một thây_ma đẹp_trai với vóc_dáng lạ_thường đã hai năm kể từ khi anh ta sống lại như một thây_ma và bị mất trí_nhớ anh cố_gắng hoàn_thiện cách nói và cách đi_đứng từ một zombie vụng_về và làm lại cuộc_đời với tư_cách là một thám_tử park_joo hyun sẽ đảm_nhận vai gong seon ji một người viết tin của một chương_trình điều_tra cô có tinh_thần lạc_quan kiên_cường và luôn muốn đấu_tranh vì công_lý khi một nhân_chứng của vụ án mà cô đang điều_tra bị tấn_công bởi một kẻ không rõ danh_tính cô đã rời ngành điều_tra sau đó cô gặp thám_tử zombie kim moo young và bắt_đầu làm_việc tại văn_phòng của anh ta thám_tử zombie zombie sẽ được sản_xuất bởi đội_ngũ nhà_sản_xuất các bộ phim the producers go back couple được đạo_diễn bởi shim_jae hyun đồng_đạo diễn của the producers và được viết bởi baek eun jin'

In [25]:
labeling([a],labels, keywords,thresholds,except_keywords)

,Chính trị quôc tế,Chủ quyền,Luật pháp - tố tụng,Nhân vật chính trị,Sai phạm,Tôn giáo - tín ngưỡng,Tư tưởng tiêu cực - chống phá nhà nước,Tổ chức nhà nước,Tổ chức nhân sự,Y tế,khác,Đất đai
0,0,0,1,0,0,0,0,0,0,0,0,0
